# Genome Assembly 1
Guided Assembly vs. De Novo Assembly

This notebook demonstrates various approaches to assembling genomes from raw reads. First, we’ll perform a guided assembly using recovered Illumina reads from a metagenomic study, specifically assembling begomovirus genomes. Second, we’ll conduct a de novo assembly without a reference.

##Install dependencies and tools##

**Install miniconda**

In [ ]:
# @title
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge

**Install fastqc, trim_galore, bowtie2, samtools, datasets, spades, pilon and quast**

In [ ]:
# @title
!conda install bioconda::fastqc -y
!!conda install trim-galore -y
!conda install bioconda::bowtie2 -y
!conda install bioconda::samtools -y
!conda install -c conda-forge ncbi-datasets-cli -y
!conda install bioconda::spades -y
!conda install bioconda::pilon -y
!conda install bioconda::quast -y

# Assisted Assembly Using a Reference
Perform a guided assembly using recovered Illumina reads from a metagenomic study, focusing specifically on assembling begomovirus genomes.

The FASTQ files contain the reads classified as begomovirus. We will focus exclusively on these reads for our analysis."

In [ ]:
!wget https://raw.githubusercontent.com/PlantHealth-Analytics/Genome_assembly/main/field_1.fastq
!wget https://raw.githubusercontent.com/PlantHealth-Analytics/Genome_assembly/main/field_2.fastq

**Run Fastqc for quality control**

In [ ]:
!fastqc *.fastq

**Check results**
Write in the blan space the name of the file the .html extension. Disply in full screen. and then esc to come back to the notebook

In [ ]:
import os
from IPython.core.display import display, HTML

# Ask the user for the file name they want to display
file_name = input("Enter the name of the HTML file you want to display (include .html extension): ")

# Check if the file exists
if os.path.exists(file_name):
    # Open and read the HTML file
    with open(file_name, 'r') as file:
        html_content = file.read()
        display(HTML(html_content))  # Display the HTML content
else:
    print(f"File '{file_name}' not found. Please ensure the file exists in the current directory.")


**Remove adapters and filter bad quality reads q > 20**

trim-galore will do the job

In [ ]:
!trim_galore --paired field_1.fastq field_2.fastq --clip_R1 15 --clip_R2 15

**Run QC to the new validate reads. XXX_val_X.fq**

In [ ]:
!fastqc *.fq

**Check the quality control results**

In [ ]:
import os
from IPython.core.display import display, HTML

# Ask the user for the file name they want to display
file_name = input("Enter the name of the HTML file you want to display (include .html extension): ")

# Check if the file exists
if os.path.exists(file_name):
    # Open and read the HTML file
    with open(file_name, 'r') as file:
        html_content = file.read()
        display(HTML(html_content))  # Display the HTML content
else:
    print(f"File '{file_name}' not found. Please ensure the file exists in the current directory.")

**Align the Reads to a Reference Genome to Conduct a Guided Assembly by Mappi**ng

Check the available complete genomes in NCBI and download them to use as guide reference genomes.


In [ ]:
!datasets summary genome taxon "East African cassava mosaic virus" --assembly-level complete \
--as-json-lines | dataformat tsv genome --fields accession,organism-name,annotinfo-name
!datasets download genome accession GCF_000859785.1 --include gff3,rna,cds,protein,genome,seq-report
!unzip ncbi_dataset.zip

Copy the genome sequence in your current folder

In [11]:
!cp ncbi_dataset/data/GCF_000859785.1/GCF_000859785.1_ViralMultiSegProj15177_genomic.fna ./

**Format the Reference Genome for Use as a Guide**

In [ ]:
!bowtie2-build GCF_000859785.1_ViralMultiSegProj15177_genomic.fna reference_genome

**Map the reads over the reference**

Mapping results are in field_isolated.sam

In [ ]:
!bowtie2 -x reference_genome -1 field_1_val_1.fq -2 field_2_val_2.fq -S field_isolated.sam

**Transform the results in sam format to bam format**

These comands will transform field_isolated.sam in field_isolated.bam, a compacted version

In [17]:
!samtools view -bS field_isolated.sam > field_isolated.bam
!samtools sort field_isolated.bam -o field_isolated.bam
!samtools index field_isolated.bam

**Create a consensus and produce and assembled molecule**

The name of the assembled genome is field_isolated_consensus.fasta

In [ ]:
#!pilon --help
!pilon --genome GCF_000859785.1_ViralMultiSegProj15177_genomic.fna --frags field_isolated.bam --output field_isolated_consensus --vcf --changes

The generated results include an assembly of your reads based on the reference genome of the cassava virus obtained from NCBI. The number of corrections or polishing steps with Pilon could be important to investigate. Performing more than one iteration is recommended until no further changes are observed. For example, the code below assembles the reads based on the reference, creates the consensus, and runs Pilon two more times for additional polishing.

Input the files

In [ ]:
# Request file names from the user
REFERENCE = input("Enter the file name for the initial reference genome (e.g., initial_assembly.fasta): ")
READS_R1 = input("Enter the file name for the first pair of reads (e.g., illumina_reads_R1.fastq): ")
READS_R2 = input("Enter the file name for the second pair of reads (e.g., illumina_reads_R2.fastq): ")

# Output prefix and other settings
OUTPUT_PREFIX = "polished_genome"  # Prefix for output files
THREADS = 2  # Number of threads for Bowtie2 and Samtools

Run the pipeline

In [ ]:
import os

# Step 1: Initial Correction using the Original Reference
print("Starting initial Pilon correction with the original reference")

# Index the original reference for Bowtie2
!bowtie2-build "$REFERENCE" reference_index

# Align reads to the original reference
!bowtie2 -x reference_index -1 "$READS_R1" -2 "$READS_R2" -S aligned_reads.sam -p {THREADS}

# Convert SAM to sorted BAM and index it
!samtools view -Sb aligned_reads.sam | samtools sort -@ {THREADS} -o sorted_reads.bam
!samtools index sorted_reads.bam

# Run Pilon with the sorted BAM file for the initial correction
initial_output = f"{OUTPUT_PREFIX}_initial"
!pilon --genome "$REFERENCE" --frags sorted_reads.bam --output "$initial_output" --changes

# Set the first corrected reference for the looped corrections
current_reference = f"{initial_output}.fasta"

# Step 2: Looped Corrections Starting with First Corrected Output
for i in range(1, 3):
    print(f"Starting Pilon iteration {i} with the corrected reference")

    # Index the current reference for Bowtie2
    !bowtie2-build "$current_reference" reference_index

    # Align reads to the current reference
    !bowtie2 -x reference_index -1 "$READS_R1" -2 "$READS_R2" -S aligned_reads.sam -p {THREADS}

    # Convert SAM to sorted BAM and index it
    !samtools view -Sb aligned_reads.sam | samtools sort -@ {THREADS} -o sorted_reads.bam
    !samtools index sorted_reads.bam

    # Run Pilon with the sorted BAM file
    pilon_output = f"{OUTPUT_PREFIX}_iter_{i}"
    !pilon --genome "$current_reference" --frags sorted_reads.bam --output "$pilon_output" --changes

    # Update the reference for the next iteration
    current_reference = f"{pilon_output}.fasta"

print("Pilon polishing complete after initial correction and 2 additional iterations.")


#*De Novo* Assembly#
De novo assembly pieces together DNA fragments to form contigs that represent an organism's chromosomes. This approach assembles a genome from scratch, without relying on a reference genome or prior knowledge of the DNA sequence.

Let’s conduct a de novo assembly using SPAdes with the metagenome option.

Run the SPAdes assembler with the metagenome (--meta) option. We use the --meta option because the reads may still contain sequences from more than one isolate or multiple types of viruses. However, SPAdes offers other assembly options that you can explore depending on the characteristics of your data. For example:

Standard SPAdes: Use for single-organism assemblies when contamination is minimal.

Plasmid SPAdes (--plasmid): Optimized for plasmid assembly.
Hybrid SPAdes (--trusted-contigs): Combines long-read and short-read data for improved assembly.
RNA-Seq SPAdes (--isolate): This flag is highly recommended for high-coverage isolate and multi-cell Illumina data; improves the assembly quality and running time

Feel free to experiment with these options to find the best fit for your dataset

https://ablab.github.io/spades/

In [ ]:
!spades.py --meta -1 field_1_val_1.fq -2 field_2_val_2.fq -o field_meta_de_novo

The output of the assembly will be in the field_meta_de_novo_ folder. This folder contains several files, but you will need the contigs.fasta file. The assembly is located here.

Assess the metrics of the assembly using QUAST.

In [ ]:
!quast field_meta_de_novo/contigs.fasta

Check the report. You will find the number of contigs. Two contigs are larger than 1 Kb. These are the assemble viral genome.  

In [ ]:
!cat quast_results/latest/report.txt

#Comparision of two type of approaches
Comparing guided assembly with Pilon (using an existing reference to refine a new assembly) and de novo assembly (assembling a genome from scratch) brings both advantages and challenges. Let’s break down the pros and cons of each approach, as well as key post-analysis steps to consider for both.

Advantages and Disadvantages
1. Guided Assembly with Pilon (Reference-Guided)
Advantages:
Higher accuracy in correcting errors: Pilon leverages high-confidence reads (like Illumina) to polish assemblies, making it especially useful when using long-read assemblies (e.g., from Nanopore or PacBio) that may have higher base-calling errors.
Consistency with known reference: Using a reference genome ensures that conserved regions align well with established knowledge, making it easier to identify variant regions or perform comparative analyses.
Efficient for closely related species: When the target species is similar to the reference genome, reference-guided assembly can achieve high coverage with fewer ambiguities.
Disadvantages:
Bias towards the reference genome: Guided assembly may miss novel regions that are absent in the reference genome or over-represent reference biases, particularly in divergent regions.
Limited structural rearrangement detection: Large structural variants or genome rearrangements may be overlooked if the reference genome is heavily relied upon, as it constrains the assembly to follow reference structure.
2. De Novo Assembly
Advantages:

Discovery of novel sequences: De novo assembly can capture previously unknown or divergent regions that may not be present in any reference, which is critical for studying novel organisms or highly variable genomic regions.
More flexible structure: It provides a comprehensive picture of the genome without bias toward any reference, allowing for detection of structural variants, unique regions, and species-specific features.
Applicability to highly divergent species: When the target organism is distantly related to available references, de novo assembly avoids reference bias, allowing a more authentic genome reconstruction.
Disadvantages:

Higher computational and data demands: De novo assembly typically requires more computational resources and high-coverage sequencing data, especially for large genomes.
Increased assembly errors in repetitive regions: Without a reference to guide repetitive regions, it can be difficult to resolve these areas accurately, leading to fragmented or misassembled contigs.
Lower accuracy without polishing: De novo assemblies, especially those using long-read technologies, often require extensive polishing with high-confidence reads to achieve high accuracy.


# "Hybrid Pipeline for Viral Genome Assembly from Metagenomic Data: Integrating SPAdes and Pilon for Enhanced Accuracy"

Description:
This pipeline provides a streamlined, hybrid approach for assembling viral genomes directly from metagenomic data. Initially, SPAdes is employed in metagenomic mode to construct the preliminary assembly, leveraging its capacity to handle complex, mixed microbial communities. Following assembly, the draft genome undergoes refinement with Pilon, using high-quality short reads to polish and correct sequencing errors.

In [ ]:
# Request file names from the user
READS_R1 = input("Enter the file name for the first pair of reads (e.g., illumina_reads_R1.fastq): ")
READS_R2 = input("Enter the file name for the second pair of reads (e.g., illumina_reads_R2.fastq): ")
OUTPUT_PREFIX = "polished_genome"  # Prefix for output files
THREADS = 2  # Number of threads for SPAdes and Samtools

In [ ]:
import os

# Step 1: Run SPAdes Assembly
print("Starting de novo assembly with SPAdes")

!spades.py --meta -1 "$READS_R1" -2 "$READS_R2" -o spades_output --threads $THREADS

# Set the initial reference for Pilon as the SPAdes contigs output
REFERENCE = "spades_output/contigs.fasta"


# Step 1: Initial Correction using the Original Reference
print("Starting initial Pilon correction with the original reference")

# Index the original reference for Bowtie2
!bowtie2-build "$REFERENCE" reference_index

# Align reads to the original reference
!bowtie2 -x reference_index -1 "$READS_R1" -2 "$READS_R2" -S aligned_reads.sam -p {THREADS}

# Convert SAM to sorted BAM and index it
!samtools view -Sb aligned_reads.sam | samtools sort -@ {THREADS} -o sorted_reads.bam
!samtools index sorted_reads.bam

# Run Pilon with the sorted BAM file for the initial correction
initial_output = f"{OUTPUT_PREFIX}_initial"
!pilon --genome "$REFERENCE" --frags sorted_reads.bam --output "$initial_output" --changes

# Set the first corrected reference for the looped corrections
current_reference = f"{initial_output}.fasta"

# Step 2: Looped Corrections Starting with First Corrected Output
for i in range(1, 3):
    print(f"Starting Pilon iteration {i} with the corrected reference")

    # Index the current reference for Bowtie2
    !bowtie2-build "$current_reference" reference_index

    # Align reads to the current reference
    !bowtie2 -x reference_index -1 "$READS_R1" -2 "$READS_R2" -S aligned_reads.sam -p {THREADS}

    # Convert SAM to sorted BAM and index it
    !samtools view -Sb aligned_reads.sam | samtools sort -@ {THREADS} -o sorted_reads.bam
    !samtools index sorted_reads.bam

    # Run Pilon with the sorted BAM file
    pilon_output = f"{OUTPUT_PREFIX}_iter_{i}"
    !pilon --genome "$current_reference" --frags sorted_reads.bam --output "$pilon_output" --changes

    # Update the reference for the next iteration
    current_reference = f"{pilon_output}.fasta"

print("Pilon polishing complete after initial correction and 3 additional iterations.")